In [5]:
import os
import subprocess
import multiprocessing
import itertools
import pandas as pd
import ast
from scipy import stats
%matplotlib inline

experiments = [
    {
        'name': 'blur-big-images',
        'filters': ['blur'],
        'languages': ['c', 'asm1', 'asm2'],
        'runs': 1000,
        'params': {}
    }
]

command = {
    'blur': './runTests {language} {filter} "{src}" "{dst}"',
    'merge': './runTests {language} {filter} "{src}" "{src2}" "{dst}" {alpha}',
    'hsl': './runTests {language} {filter} "{src}" "{dst}" {h} {s} {l}'
}

trim = 0.25

def runExperiment(filter, arguments):
    comm = command[filter].format(**arguments)
    delta = subprocess.check_output(comm, shell=True)
    delta = ast.literal_eval(delta.strip())
    rdelta = pd.Series(delta)
    
    return {
            'delta': delta,
            'mean': rdelta.mean(),
            'var': rdelta.var(),
            'tmean': stats.trim_mean(rdelta, trim),
            'min': rdelta.min()
        }

def experimentWorker(experiment, param, dataset, procImages):
    output = []
    
    for (f, l, src) in itertools.product(experiment['filters'], experiment['languages'], procImages):
        expOutput = {
            'experiment': experiment['name'],
            'make-param': param,
            'filename': os.path.splitext(os.path.basename(src))[0],
            'filter': f,
            'language': l
        }
        
        expOutput['size'] = expOutput['filename'].split(' ')[1].split('x')[0]

        arguments = {
            'language': l,
            'filter': f,
            'src': src,
            'dst': '/dev/null'
        }
        
        if f == 'hsl':
            for (h, s, l) in experiment['params'][f]:
                pars = {
                        'h': h,
                        's': s,
                        'l': l
                    }

                args = arguments.copy()
                args.update(pars)
                args = runExperiment(f, args)
                args.update(pars)
                args.update(expOutput)
                output.append(args)
        elif f == 'merge':
            for alpha in experiment['params'][f]:
                pars = {
                        'alpha': alpha,
                        'src2': src
                    }

                args = arguments.copy()
                args.update(pars)
                args = runExperiment(f, args)
                args['alpha'] = alpha
                args.update(expOutput)
                output.append(args)
        else:
            args = runExperiment(f, arguments)
            args.update(expOutput)
            output.append(args)

    dataset += output
    del output

manager = multiprocessing.Manager()
dataset = manager.list([])

base_path = '/home/julian/Development/orga2-tp2'
src_path = os.path.join(base_path, 'src')
filters_path = os.path.join(src_path, 'filters')
test_path = os.path.join(src_path, 'tests')
makeparams = ['o3fastshuffles']
# ['o0shift', 'o0shuffles', 'o3shuffles', 'o3shift', 'o3fastshift']
exp_img_path = os.path.join(base_path, 'exp', 'images')

for param in makeparams:
    subprocess.check_output('make -C "' + filters_path + '" clean', executable='/bin/bash', shell=True)
    subprocess.check_output('make -C "' + test_path + '" clean' , executable='/bin/bash', shell=True)
    subprocess.check_output('make -C "' + filters_path + '" ' + param, executable='/bin/bash', shell=True)
    subprocess.check_output('make -C "' + test_path + '"', executable='/bin/bash', shell=True)
    
    jobs = []

    for experiment in experiments:
        images_path = os.path.join(exp_img_path, experiment['name'])

        procImages = []
        
        for img in os.listdir(images_path):
            rpath = os.path.join(images_path, img)

            if os.path.isfile(rpath):
                procImages.append(rpath)

        p = multiprocessing.Process(target=experimentWorker, args=(experiment, param, dataset, procImages))
        jobs.append(p)
        p.start()
    
    for job in jobs:
        job.join()

In [7]:
ds = []

for x in dataset:
    ds.append(x)
    
ds = pd.DataFrame(ds)

In [12]:
ds.to_csv('bigBlurdataset.csv')

In [10]:
ds['size'] = ds['filename'].apply(lambda x: x.split(' ')[0].split('x')[0])

In [11]:
ds

,delta,experiment,filename,filter,language,make-param,mean,min,size,tmean,var
0,"[236985077, 238286481, 238316954, 236030521, 2...",blur-big-images,2972x2972 (1939751544),blur,c,o3fastshuffles,2.392552e+08,234973751,2972,2.380471e+08,1.594418e+13
1,"[234661546, 234657306, 234486237, 234639854, 2...",blur-big-images,2956x2956 (1544950461),blur,c,o3fastshuffles,2.347335e+08,230845313,2956,2.344652e+08,3.702292e+12
2,"[245555830, 234171740, 232035833, 235163358, 2...",blur-big-images,2968x2968 (226048380),blur,c,o3fastshuffles,2.339307e+08,231629049,2968,2.335112e+08,3.752917e+12
3,"[264738067, 232350109, 232261610, 232403720, 2...",blur-big-images,2964x2964 (1760711477),blur,c,o3fastshuffles,2.334706e+08,231057495,2964,2.328355e+08,1.230684e+13
4,"[235736318, 232488776, 233498996, 233526984, 2...",blur-big-images,2960x2960 (406921733),blur,c,o3fastshuffles,2.346833e+08,232397065,2960,2.341900e+08,4.278226e+12
5,"[157618616, 143663166, 142931993, 141341290, 1...",blur-big-images,2972x2972 (1939751544),blur,asm1,o3fastshuffles,1.435715e+08,140971029,2972,1.429390e+08,6.080891e+12
6,"[151688341, 140226852, 140896736, 140572866, 1...",blur-big-images,2956x2956 (1544950461),blur,asm1,o3fastshuffles,1.413207e+08,139206940,2956,1.409090e+08,3.004214e+12
7,"[143440843, 139257207, 141352594, 140630615, 1...",blur-big-images,2968x2968 (226048380),blur,asm1,o3fastshuffles,1.417375e+08,138969034,2968,1.413164e+08,4.145755e+12
8,"[142382896, 141847299, 141613366, 139467199, 1...",blur-big-images,2964x2964 (1760711477),blur,asm1,o3fastshuffles,1.416818e+08,139274765,2964,1.410621e+08,5.913764e+12
9,"[143247547, 143758252, 142375336, 142642324, 1...",blur-big-images,2960x2960 (406921733),blur,asm1,o3fastshuffles,1.406005e+08,137902627,2960,1.404508e+08,2.731654e+12
